<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment17/assignment17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing all python packages

In [1]:
# Ensure latest TensorFlow is installed.
!pip install -q tf-nightly-gpu-2.0-preview
# Load the TensorBoard notebook extension.
%load_ext tensorboard

     |████████████████████████████████| 392.0MB 55kB/s 
     |████████████████████████████████| 450kB 35.4MB/s 
     |████████████████████████████████| 3.8MB 23.9MB/s 


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import tensorflow as tf
#tf.enable_eager_execution()

import functools
import tensorflow_datasets as tfds
from tensorflow.python.keras import backend
from tensorflow.python.keras import layers
import numpy as np

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.0.0-dev20190920


In [3]:
tf.executing_eagerly()

True

In [0]:
from google.colab import drive
import time, math

In [0]:
#import numpy as np
#import time, math
#from tqdm import tqdm_notebook as tqdm

#import tensorflow as tf
#import tensorflow.contrib.eager as tfe

#from google.colab import drive
#from datetime import datetime

**clone the utility repo created**

In [5]:
!rm -rf DNN_utils
!git clone https://github.com/ApoorvaNagarajan/DNN_utils.git

from DNN_utils import tfrecord_utils
from DNN_utils import data_transforms as dt

Cloning into 'DNN_utils'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 105 (delta 17), reused 0 (delta 0), pack-reused 66
Receiving objects: 100% (105/105), 46.47 KiB | 5.81 MiB/s, done.
Resolving deltas: 100% (40/40), done.


Using TensorFlow backend.


**Mount google drive to fetch the CIFAR dataset stored in tfRecord format**

In [6]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


**Configuration**

In [0]:
# Hyperparameters

BATCH_SIZE = 512 #@param {type:"number"}
EPOCHS = 10 #@param {type:"number"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}




In [0]:
#Configuration

NUM_CLASSES = 10 
IMG_SHAPE=[32, 32, 3]
NUM_TRAIN_IMG = 50000
NUM_TEST_IMG = 10000
BATCHES_PER_EPOCH = NUM_TRAIN_IMG//BATCH_SIZE
VALIDATION_STEPS = NUM_TEST_IMG//BATCH_SIZE

#Load tfRecord format CIFAR10 dataset

In [0]:
tfrecord_utils.convert_to_tfRecord('CIFAR10',trainFile='/content/drive/My Drive/datasets/cifarTfTrain.tfrecords', testFile='/content/drive/My Drive/datasets/cifarTfTest.tfrecords')
#train_x, train_y = tfrecord_utils.parse_tfRecord('/content/drive/My Drive/datasets/cifarTfTrain.tfrecords', NUM_TRAIN_IMG, BATCH_SIZE, IMG_SHAPE, NUM_CLASSES)
#test_x, test_y = tfrecord_utils.parse_tfRecord('/content/drive/My Drive/datasets/cifarTfTest.tfrecords', NUM_TEST_IMG, BATCH_SIZE, IMG_SHAPE, NUM_CLASSES)
train_set = tfrecord_utils.get_dataset('/content/drive/My Drive/datasets/cifarTfTrain.tfrecords',IMG_SHAPE,NUM_CLASSES)
test_set = tfrecord_utils.get_dataset('/content/drive/My Drive/datasets/cifarTfTest.tfrecords',IMG_SHAPE,NUM_CLASSES)

#Define model

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))

In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, inputs):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(inputs)))))
    h = self.linear(h) * self.weight
    h = tf.nn.softmax(h)
    return h

#Create network and run

In [0]:
model = DavidNet()

#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
#lr_func = lambda: lr_schedule(EPOCHS)/BATCH_SIZE
#opt = tf.compat.v1.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (dt.random_pad_crop(dt.randomFlip(dt.cutOut(dt.rotate(x, 10),8,8,0)), 4),y)
#data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)

In [0]:
#import IPython.display as display
#import matplotlib.pyplot as plt
#import numpy as np
#% matplotlib inline

#for parsed_record in train_set.take(10):
#  print(repr(parsed_record))

#for record in train_set.take(10):
#  print('here')
#  plt.rcParams['figure.figsize'] = (1,1)
#  f, ax = plt.subplots(1, 1)
#  ax.set_xticks([])
#  ax.set_yticks([])
#  ax.imshow(record[0].numpy().reshape(32,32,3).astype('int32'))
#  break

In [0]:
log_dir="logs/profile/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = 3)


In [0]:
def scheduler(epoch, lr):
  return np.interp(epoch, [0, (EPOCHS+1)//5, EPOCHS], [0.01, LEARNING_RATE, 0])


In [0]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
train_set = train_set.repeat()
train_set_aug = train_set.map(data_aug).shuffle(NUM_TRAIN_IMG).batch(BATCH_SIZE)
train_set = train_set.prefetch(1)
test_set = test_set.repeat()
test_set = test_set.batch(BATCH_SIZE)

model.fit(train_set_aug,
          steps_per_epoch=BATCHES_PER_EPOCH,
          epochs=EPOCHS, 
          callbacks=[tensorboard_callback,tf.keras.callbacks.LearningRateScheduler(scheduler)],
          validation_data = test_set,
          validation_steps = VALIDATION_STEPS)

Train for 97 steps, validate for 19 steps
Epoch 1/10
97/97 [==============================] - 136s 1s/step - loss: 2.1832 - accuracy: 0.2135 - val_loss: 375.5044 - val_accuracy: 0.1293
Epoch 2/10
97/97 [==============================] - 96s 994ms/step - loss: 1.8753 - accuracy: 0.2994 - val_loss: 649.7538 - val_accuracy: 0.1001
Epoch 3/10
97/97 [==============================] - 96s 994ms/step - loss: 1.8254 - accuracy: 0.3353 - val_loss: 447.2818 - val_accuracy: 0.1072
Epoch 4/10
97/97 [==============================] - 96s 993ms/step - loss: 1.5684 - accuracy: 0.4274 - val_loss: 442.3229 - val_accuracy: 0.1103
Epoch 5/10
97/97 [==============================] - 97s 995ms/step - loss: 1.4218 - accuracy: 0.4811 - val_loss: 308.7060 - val_accuracy: 0.1352
Epoch 6/10
97/97 [==============================] - 96s 985ms/step - loss: 1.2867 - accuracy: 0.5320 - val_loss: 375.3914 - val_accuracy: 0.1161
Epoch 7/10
97/97 [==============================] - 96s 994ms/step - loss: 1.1704 - accura

In [53]:
!tar -zcvf logs.tar.gz logs/profile/

logs/profile/
logs/profile/20190920-173117/
logs/profile/20190920-173117/train/
logs/profile/20190920-173117/train/events.out.tfevents.1569000766.9cb675ed8ddd.profile-empty
logs/profile/20190920-173117/train/plugins/
logs/profile/20190920-173117/train/plugins/profile/
logs/profile/20190920-173117/train/plugins/profile/2019-09-20_17-32-46/
logs/profile/20190920-173117/train/plugins/profile/2019-09-20_17-32-46/local.trace
logs/profile/20190920-173117/train/events.out.tfevents.1569000723.9cb675ed8ddd.123.21913.v2
logs/profile/20190920-173117/validation/
logs/profile/20190920-173117/validation/events.out.tfevents.1569000859.9cb675ed8ddd.123.24273.v2


In [0]:
!rm -rf logs